<a href="https://colab.research.google.com/github/ennioguzman91/Foundations-of-Computer-Science/blob/main/Progetto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You have to work on the University dataset. It contains three different university rankings:

* The Times Higher Education World University Ranking, shortly Times,
* the Academic Ranking of World Universities, shortly Shanghai,
* the Center for World University Rankings, shortly cwur.

Notes
1. It is mandatory to use GitHub for developing the project.
2. The project must be a jupyter notebook.
3. There is no restriction on the libraries that can be used, nor on the Python version
4. All questions on the project must be asked in a public channel on Zulip.

In [131]:
# Import all libraries
import pandas as pd
import numpy as np
import functools

In [89]:
from google.colab import drive
drive.mount('/content/drive')
dir_files = '/content/drive/MyDrive/Università/Data-Science-Bicocca/Corsi/Foundations-of-Computer-Science/Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
# Import the files
times_df = pd.read_csv(dir_files + 'timesData.csv')
shanghai_df = pd.read_csv(dir_files + 'shanghaiData.csv')
cwur_df = pd.read_csv(dir_files + 'cwurData.csv')

1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes¶

In [91]:
times_df.tail()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2598,601-800,Yeungnam University,South Korea,18.6,24.3,10.9,26.5,35.4,-,"21,958",15.3,3%,48 : 52,2016
2599,601-800,Yıldız Technical University,Turkey,14.5,14.9,7.6,19.3,44.0,-,"31,268",28.7,2%,36 : 64,2016
2600,601-800,Yokohama City University,Japan,24.0,16.1,10.2,36.4,37.9,-,"4,122",3.7,3%,NaN,2016
2601,601-800,Yokohama National University,Japan,20.1,23.3,16.0,13.5,40.4,-,"10,117",12.1,8%,28 : 72,2016
2602,601-800,Yuan Ze University,Taiwan,16.2,17.7,18.3,28.6,39.8,-,"8,663",20.6,4%,43 : 57,2016


In [92]:
# times_df.head()
times_df[times_df['university_name'] == 'Harvard University']

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
201,2,Harvard University,United States of America,95.8,67.5,97.4,99.8,35.9,93.9,"20,152",8.9,25%,NaN,2012
605,4,Harvard University,United States of America,94.9,63.7,98.6,99.2,39.9,93.6,"20,152",8.9,25%,NaN,2013
1003,2,Harvard University,United States of America,95.3,66.2,98.5,99.1,40.6,93.9,"20,152",8.9,25%,NaN,2014
1403,2,Harvard University,United States of America,92.9,67.6,98.6,98.9,44.0,93.3,"20,152",8.9,25%,NaN,2015
1808,6,Harvard University,United States of America,83.6,77.2,99.0,99.8,45.2,91.6,"20,152",8.9,25%,NaN,2016


In [93]:
len(times_df['university_name'].unique())

818

In [94]:
filtered_min_year = times_df.loc[ times_df.groupby('university_name')['year'].idxmin()  ]
filtered_min_year
# filtered_min[ filtered_min['university_name'] =='Harvard University'] 

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,42,École Normale Supérieure,France,66.8,44.9,48.2,95.7,30.7,68.6,"2,400",7.9,20%,46 : 54,2011
99,100,École Normale Supérieure de Lyon,France,51.1,37.6,34.4,88.8,26.1,57.0,"2,218",8.0,14%,49 : 51,2011
38,39,École Polytechnique,France,57.9,77.9,56.1,91.4,-,69.5,"2,429",4.8,30%,18 : 82,2011
47,48,École Polytechnique Fédérale de Lausanne,Switzerland,55.0,100.0,56.1,83.8,38.0,66.5,"9,666",10.5,54%,27 : 73,2011


In [95]:
filtered_max_year = times_df.loc[ times_df.groupby('university_name')['year'].idxmax() ]
filtered_max_year

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,54,École Normale Supérieure,France,70.6,85.5,47.7,87.1,37.1,69.0,"2,400",7.9,20%,46 : 54,2016
2013,201-250,École Normale Supérieure de Lyon,France,41.6,65.6,30.0,69.0,31.7,-,"2,218",8.0,14%,49 : 51,2016
1904,=101,École Polytechnique,France,53.5,92.8,44.6,64.7,82.3,57.9,"2,429",4.8,30%,18 : 82,2016
1833,31,École Polytechnique Fédérale de Lausanne,Switzerland,61.3,98.6,67.5,94.6,65.4,76.1,"9,666",10.5,54%,27 : 73,2016


2. For each university, compute the improvement in income between the least recent and the most recent data points

In [96]:
pd.options.display.max_rows = None

# from the dataframe to a serie with only the income
# use the university name as index

data_exe_2_max = filtered_max_year.set_index('university_name')['income']
data_exe_2_min = filtered_min_year.set_index('university_name')['income']

# Set to 0 the values that doesnt have the two income values

data_exe_2_max[ data_exe_2_min == "-"  ] = '-' 
data_exe_2_min[ data_exe_2_max == "-"  ] = '-'
data_exe_2_max = data_exe_2_max.replace('-', 0)
data_exe_2_min = data_exe_2_min.replace('-', 0)


pd.to_numeric(data_exe_2_max) - pd.to_numeric(data_exe_2_min)



university_name
AGH University of Science and Technology                               0.0
Aalborg University                                                     7.3
Aalto University                                                      -0.3
Aarhus University                                                      6.8
Aberystwyth University                                                -4.2
Adam Mickiewicz University                                             0.0
Aix-Marseille University                                               0.0
Ajou University                                                        0.0
Alexandria University                                                 -6.3
Alexandru Ioan Cuza University                                         0.0
Aligarh Muslim University                                              0.0
American University                                                    0.0
American University of Beirut                                          0.0
American 

3. Find the university with the largest increase computed in the previous point

In [97]:
largest_increase_db = pd.to_numeric(data_exe_2_max) - pd.to_numeric(data_exe_2_min)
largest_increase_db.idxmax()

'TU Dresden'

In [98]:
largest_increase_db.loc[largest_increase_db.idxmax()] 

67.80000000000001

4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range¶

In [99]:
times_df[times_df['university_name'] == 'Aarhus University']

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
324,125,Aarhus University,Denmark,30.7,69.3,49.0,64.0,67.0,50.0,"23,895",13.6,14%,54 : 46,2012
717,116,Aarhus University,Denmark,35.7,66.4,54.6,71.6,69.7,55.3,"23,895",13.6,14%,54 : 46,2013
1139,138,Aarhus University,Denmark,30.7,67.2,46.4,65.8,67.5,49.6,"23,895",13.6,14%,54 : 46,2014
1554,153,Aarhus University,Denmark,30.9,66.3,47.5,66.5,59.8,49.9,"23,895",13.6,14%,54 : 46,2015
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016


In [100]:
"=200".split("-")[-1].replace("=","")

'200'

In [101]:
pd.options.display.max_rows = 400
# cleaning the ranking data
ranking_db = times_df[['world_rank', 'university_name', 'year']]
ranking_db['world_rank_cleaned'] = list(map( lambda x: int(x.split("-")[0].replace("=","")), ranking_db['world_rank']  ))
ranking_db

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,world_rank,university_name,year,world_rank_cleaned
0,1,Harvard University,2011,1
1,2,California Institute of Technology,2011,2
2,3,Massachusetts Institute of Technology,2011,3
3,4,Stanford University,2011,4
4,5,Princeton University,2011,5
...,...,...,...,...
2598,601-800,Yeungnam University,2016,601
2599,601-800,Yıldız Technical University,2016,601
2600,601-800,Yokohama City University,2016,601
2601,601-800,Yokohama National University,2016,601


In [102]:
ranking_db_min = ranking_db.loc[ ranking_db.groupby('university_name')['world_rank_cleaned'].idxmin()  ]
ranking_db_min[ranking_db_min['university_name'] == 'Aarhus University' ]

,world_rank,university_name,year,world_rank_cleaned
1908,=106,Aarhus University,2016,106


In [103]:
pd.options.display.max_rows = 50
ranking_db_max = ranking_db.loc[ ranking_db.groupby('university_name')['world_rank_cleaned'].idxmax()  ]
ranking_db_max[ranking_db_max['university_name'] == 'Aarhus University' ]

,world_rank,university_name,year,world_rank_cleaned
166,167,Aarhus University,2011,167


In [104]:
pd.options.display.max_rows = 50

ranking_db_total = pd.DataFrame( index = ranking_db_max['university_name'].tolist() )
ranking_db_total['rank_max'] = ranking_db_max.set_index('university_name')['world_rank_cleaned']
ranking_db_total['rank_min'] = ranking_db_min.set_index('university_name')['world_rank_cleaned']
ranking_db_total.loc['Aarhus University' ]

ranking_db_total['Diff'] = ranking_db_total['rank_max']-ranking_db_total['rank_min']
ranking_db_total


,rank_max,rank_min,Diff
AGH University of Science and Technology,601,601,0
Aalborg University,351,201,150
Aalto University,301,251,50
Aarhus University,167,106,61
Aberystwyth University,351,276,75
...,...,...,...
École Normale Supérieure,78,42,36
École Normale Supérieure de Lyon,201,100,101
École Polytechnique,101,39,62
École Polytechnique Fédérale de Lausanne,48,31,17


5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.¶

In [117]:
pd.options.display.max_rows = 50

# cleaning the nan data
ratio_woman_male_db = times_df[['university_name', 'year', 'female_male_ratio', 'num_students']].replace('-', np.nan).dropna()
ratio_woman_male_db = ratio_woman_male_db.loc[ ratio_woman_male_db.groupby('university_name')['year'].idxmax()  ]

#processing the ratio

ratio_woman_male_db['female_male_ratio']
ratio_woman_male_db['num_students'] = list( map( lambda x: int( x.replace(',','')), ratio_woman_male_db['num_students']  )) # procesing num_students
ratio_woman_male_db['male'] = list(map( lambda x,y: round( y/100 * int(x.split(" : ")[-1]) ), ratio_woman_male_db['female_male_ratio'], ratio_woman_male_db['num_students']  )) # from ratio -> males
ratio_woman_male_db['female'] = ratio_woman_male_db['num_students'] - ratio_woman_male_db['male'] # obtaning females


ratio_woman_male_db



,university_name,year,female_male_ratio,num_students,male,female
2003,Aalborg University,2016,48 : 52,17422,9059,8363
2056,Aalto University,2016,32 : 68,16099,10947,5152
1908,Aarhus University,2016,54 : 46,23895,10992,12903
2105,Aberystwyth University,2016,48 : 52,9252,4811,4441
2404,Adam Mickiewicz University,2016,71 : 29,40633,11784,28849
...,...,...,...,...,...,...
1856,École Normale Supérieure,2016,46 : 54,2400,1296,1104
2013,École Normale Supérieure de Lyon,2016,49 : 51,2218,1131,1087
1904,École Polytechnique,2016,18 : 82,2429,1992,437
1833,École Polytechnique Fédérale de Lausanne,2016,27 : 73,9666,7056,2610


6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)¶

In [154]:

ratio_woman_male_mean_6_db = times_df[['university_name', 'year', 'female_male_ratio', 'country', 'num_students']].replace('-', np.nan).dropna()
ratio_woman_male_mean_6_db = ratio_woman_male_mean_6_db.loc[ ratio_woman_male_mean_6_db.groupby('university_name')['year'].idxmax()  ]
ratio_woman_male_mean_6_db['female'] = list(map( lambda x: int(x.split(" : ")[0]), ratio_woman_male_mean_6_db['female_male_ratio']  ))
average_ratio = ratio_woman_male_mean_6_db['female'].mean() #mean of female
ration_female_6 = ratio_woman_male_mean_6_db[ ratio_woman_male_mean_6_db['female'] > average_ratio ]
ration_female_6
# ratio_woman_male_mean_6_db
# list(ratio_woman_male_mean_6_db['university_name'] )


,university_name,year,female_male_ratio,country,num_students,female
1908,Aarhus University,2016,54 : 46,Denmark,"23,895",54
2404,Adam Mickiewicz University,2016,71 : 29,Poland,"40,633",71
2057,Aix-Marseille University,2016,61 : 39,France,"71,749",61
2409,Alexandru Ioan Cuza University,2016,63 : 37,Romania,"25,724",63
2204,American University,2016,60 : 40,United States of America,"11,604",60
...,...,...,...,...,...,...
2302,Xiamen University,2016,52 : 48,China,"32,921",52
1814,Yale University,2016,50 : 50,United States of America,"11,751",50
1968,Yeshiva University,2016,53 : 47,United States of America,"6,753",53
2152,York University,2016,59 : 41,Canada,"48,007",59


7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

In [159]:
pd.options.display.max_rows = None
ration_female_7 = ration_female_6.copy()
ration_female_7['num_students'] = ration_female_7['num_students'].apply(lambda x: pd.to_numeric(x.replace(',','')) )# = ration_female_6['num_students']
total_students = functools.reduce( lambda a,b: a+b, ration_female_7['num_students']  )
# ration_female_7
ration_female_7.groupby('country')['num_students'].sum()/ total_students


country
Argentina                   0.009157
Australia                   0.051074
Austria                     0.006833
Belarus                     0.002476
Belgium                     0.014335
Brazil                      0.022730
Canada                      0.047980
China                       0.018029
Colombia                    0.003284
Cyprus                      0.000581
Czech Republic              0.011591
Denmark                     0.006223
Egypt                       0.001421
Estonia                     0.000868
Finland                     0.005608
France                      0.049729
Germany                     0.044564
Greece                      0.012236
Hong Kong                   0.006782
Hungary                     0.005519
Iceland                     0.001180
India                       0.002752
Indonesia                   0.003681
Iran                        0.002679
Israel                      0.004467
Italy                       0.087833
Japan                       0.

8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [177]:
pd.options.display.max_rows = 500
educaction_df = pd.read_csv(dir_files + 'educational_attainment_supplementary_data.csv')
educaction_df

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,NaN,NaN,NaN,NaN,0.75,NaN,NaN,NaN,NaN,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,NaN,NaN,NaN,NaN,2.01,NaN,NaN,NaN,NaN,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,NaN,NaN,NaN,NaN,1.92,NaN,NaN,NaN,NaN,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,NaN,NaN,NaN,NaN,3.83,NaN,NaN,NaN,NaN,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,NaN,NaN,NaN,NaN,1.26,NaN,NaN,NaN,NaN,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
education_8_db = educaction_df [ pd.notna(educaction_df['country_name']) & pd.notna(educaction_df['series_name'])  ]
education_8_db

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,NaN,NaN,NaN,NaN,0.75,NaN,NaN,NaN,NaN,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,NaN,NaN,NaN,NaN,2.01,NaN,NaN,NaN,NaN,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,NaN,NaN,NaN,NaN,1.92,NaN,NaN,NaN,NaN,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,NaN,NaN,NaN,NaN,3.83,NaN,NaN,NaN,NaN,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,NaN,NaN,NaN,NaN,1.26,NaN,NaN,NaN,NaN,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [234]:
education_9_db_base = education_8_db[['country_name', 'series_name']].copy()
education_9_db = education_8_db.copy()

education_9_final = pd.DataFrame(columns=['country_name', 'series_name', 'year', 'value'])

for col_num in range(2, len(list(education_9_db) )) :
  for indice in  education_9_db.index.values.tolist():
    if pd.notna( education_9_db.loc[indice, list(education_9_db)[col_num]] ):
      # print('hola')
      new_row = {
        'country_name': education_9_db_base.loc[indice, 'country_name'], 
        'series_name': education_9_db_base.loc[indice, 'series_name'],
        'year': list(education_9_db)[col_num],
        'value': education_9_db.loc[indice, list(education_9_db)[col_num]]
      }
      education_9_final = education_9_final.append(new_row, ignore_index=True)
      # print(new_row)
  #   break
  # break
# print('hola')  
# print(education_9_final)


In [235]:
education_9_final

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33000
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03000
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83000
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34000
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54000
...,...,...,...,...
318285,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.48356
318286,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.88820
318287,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000
318288,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000


10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).¶


In [245]:

number_ranking_10 = times_df[['university_name', 'world_rank', 'year' ]]
number_ranking_10.groupby('university_name').count()

,world_rank,year
university_name,,
AGH University of Science and Technology,1,1
Aalborg University,5,5
Aalto University,5,5
Aarhus University,6,6
Aberystwyth University,5,5
...,...,...
École Normale Supérieure,6,6
École Normale Supérieure de Lyon,6,6
École Polytechnique,6,6


11. In the times ranking, compute the number of times each university appears

In [249]:
numbers_11 = times_df[['university_name', 'year']]
numbers_11.groupby('university_name').count()

,year
university_name,
AGH University of Science and Technology,1
Aalborg University,5
Aalto University,5
Aarhus University,6
Aberystwyth University,5
...,...
École Normale Supérieure,6
École Normale Supérieure de Lyon,6
École Polytechnique,6


12. Find the universities that appear at most twice in the times ranking.

In [251]:
number_ranking_12 = times_df[['university_name', 'world_rank', 'year' ]]
group_ranking_12 = number_ranking_10.groupby('university_name').count()
group_ranking_12[ group_ranking_12['world_rank'] >= 2 ]

,world_rank,year
university_name,,
Aalborg University,5,5
Aalto University,5,5
Aarhus University,6,6
Aberystwyth University,5,5
Alexandria University,3,3
Arizona State University,6,6
Aston University,3,3
Auburn University,2,2
Australian National University,6,6
